# Clase Práctica 01

# Neural Network - Redes Neuronales Artificiales

# Primera Red Neuronal con Keras



# Introducción

Este tutorial muestra el flujo de trabajo básico de usar Keras con un modelo lineal simple, se va a explicar y desarrollar un código básico con Tensorflow y Keras.

Los pasos a seguir para el desarrollo del código son los siguientes: 

1. Cargar los datos.
2. Definir nuestro modelo.
3. Compilar nuestro modelo.
4. Ajustar nuestro modelo.
5. Evaluar nuestro modelo.
6. Unir todo el proceso.



# Base de datos: Pima Indians

En este tutorial vamos a utilizar la base de datos de diabetes Pima Indians. La base de datos describe los datos de los registros médicos de los pacientes que tuvieron un inicio de diabetes dentro de los cinco años. Corresponde a un problema de clasificación binaria (aparición de diabetes como 1 o no aparición como 0). Las variables de entrada que describen a cada paciente son numéricas y tienen escalas variables. La base de datos posee 768 instancias. 

A continuación se enumeran los ocho atributos (descritos en inglés) para el conjunto de datos:

1. Number of times pregnant.
2. Plasma glucose concentration a 2 hours in an oral glucose tolerance test.
3. Diastolic blood pressure (mm Hg).
4. Triceps skin fold thickness (mm).
5. 2-Hour serum insulin (mu U/ml).
6. Body mass index.
7. Diabetes pedigree function.
8. Age (years).
9. Class, onset of diabetes within five years.



# Cargar librerías y datos  

Se utiliza una semilla (seed) para hacer los experimentos de Deep Learning repetibles. Esto es útil si necesita demostrar un resultado, comparar algoritmos usando la misma fuente de aleatoriedad o depurar una parte de su código. Puede inicializar el generador de números aleatorios con cualquier semilla que desee, por ejemplo: 7 como el que usamos en el ejemplo. 

In [1]:
# librerías
import torch
from torch import nn
from torch.nn import Module
from torch.nn import Linear
from torch.nn import Sequential
from collections import OrderedDict
from torch.utils.data import DataLoader
from torch.nn import functional as F
import numpy

# seed fija para experimentos
seed = 7
numpy.random.seed(seed)

Ahora podemos cargar nuestra base de datos. Se puede cargar el archivo directamente usando la función NumPy loadtxt(). Hay ocho variables de entrada y una variable de salida (la última columna contiene las etiquetas). Una vez cargado, podemos dividir el conjunto de datos en variables de entrada (X) y la variable de clase de salida (Y).

In [2]:
# cargar los datos
dataset_numpy = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
data =  DataLoader(torch.from_numpy(dataset_numpy).float(), batch_size=10, shuffle=True)

X = torch.from_numpy(dataset_numpy[:,0:8])
Y = torch.reshape(torch.from_numpy(dataset_numpy[:,8]), (-1,1))


print("Entradas:")
print(X)
print("Salidas:")
print(Y)
print("Observe que hay 768 ejemplos y 8 atributos:")
print(len(X))
print("Observe que hay 768 etiquetas")
print((Y.shape))

Entradas:
tensor([[  6.0000, 148.0000,  72.0000,  ...,  33.6000,   0.6270,  50.0000],
        [  1.0000,  85.0000,  66.0000,  ...,  26.6000,   0.3510,  31.0000],
        [  8.0000, 183.0000,  64.0000,  ...,  23.3000,   0.6720,  32.0000],
        ...,
        [  5.0000, 121.0000,  72.0000,  ...,  26.2000,   0.2450,  30.0000],
        [  1.0000, 126.0000,  60.0000,  ...,  30.1000,   0.3490,  47.0000],
        [  1.0000,  93.0000,  70.0000,  ...,  30.4000,   0.3150,  23.0000]],
       dtype=torch.float64)
Salidas:
tensor([[1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
       

# Definir el modelo de red neuronal

Los modelos en Keras se definen como una secuencia de capas. Por lo tanto, vamos a crear un modelo secuencial y agregamos capas secuencialmente para obtener nuestra topología de red. Debemos asegurarnos que la capa de entrada que hemos definimos tenga el número correcto de entradas. Esto se puede especificar al crear la primera capa con el argumento llamado "dimensión de entrada" y establecerlo en 8 para las 8 variables de entrada.

¿Cómo sabemos el número de capas a usar y sus tipos? Esta es una pregunta muy difícil en Deep Learning. Podemos utilizar heurísticas y, a menudo, la mejor estructura de red se encuentra a través de un proceso de experimentación de prueba y error. En general, necesita una red lo suficientemente grande como para capturar la estructura del problema, si es que eso ayuda. En este ejemplo, utilizaremos una estructura de red totalmente conectada (fully connected) con tres capas.

Las capas totalmente conectadas (fully connected) se definen utilizando la clase "Dense". Podemos especificar el número de neuronas en la capa en el primer argumento; el método de inicialización de la red en el segundo argumento como init; y además, se puede especificar la función de activación utilizando el tercer argumento (función de activación). Para nuestro ejemplo, vamos a inicializar los pesos de la red de manera aleatoria, generada a partir de una distribución uniforme. Usaremos en este caso, valores entre 0 y 0.05, porque esa es la inicialización de pesos uniformes que trae de forma predeterminada Keras. Otra alternativa tradicional sería normal para los pequeños números aleatorios generados a partir de una distribución gaussiana.

Usaremos la función de activación de rectificación (ReLU) en las dos primeras capas y la función de activación sigmoide en la capa de salida. Tambien es posible usar las funciones de activación sigmoide y tanh para todas las capas. Actualente, se observa un mejor rendimiento (empírico) utilizando la función de activación de rectificación ReLU. Utilizamos una función de activación sigmoidea en la capa de salida para garantizar que nuestra salida de red esté entre 0 y 1, y sea fácil de asignar a una probabilidad. La primera capa oculta tiene 12 neuronas y espera 8 variables de entrada. La segunda capa oculta tiene 8 neuronas y solo la capa de salida tiene 1 neurona para predecir la clase (aparición de diabetes o no).

<img src="./images_tutoriales/Imagen1.png">


In [20]:
help(Linear)

Help on class Linear in module torch.nn.modules.linear:

class Linear(torch.nn.modules.module.Module)
 |  Linear(in_features: int, out_features: int, bias: bool = True, device=None, dtype=None) -> None
 |  
 |  Applies a linear transformation to the incoming data: :math:`y = xA^T + b`
 |  
 |  This module supports :ref:`TensorFloat32<tf32_on_ampere>`.
 |  
 |  Args:
 |      in_features: size of each input sample
 |      out_features: size of each output sample
 |      bias: If set to ``False``, the layer will not learn an additive bias.
 |          Default: ``True``
 |  
 |  Shape:
 |      - Input: :math:`(*, H_{in})` where :math:`*` means any number of
 |        dimensions including none and :math:`H_{in} = \text{in\_features}`.
 |      - Output: :math:`(*, H_{out})` where all but the last dimension
 |        are the same shape as the input and :math:`H_{out} = \text{out\_features}`.
 |  
 |  Attributes:
 |      weight: the learnable weights of the module of shape
 |          :math:`(

In [3]:
model = Sequential(
        nn.Linear(8,12),
        nn.ReLU(),
        nn.Linear(12,8),
        nn.ReLU(),
        nn.Linear(8,1)
        )

# queda pendiente inicializar los pesos con alguna funcion,
# hasta ahora al aplicar la funcion de inicializacion
# hace que la perdida sea muy alta

In [7]:
#estilo pytorch
# Las redes (modelos en general) deben heredar desde torch.nn.Module
# modelo secuencial
class FFNN(Module):
  
  #definición 
  def __init__(self, d0=8, d1=12):
    super(FFNN, self).__init__()
    
    # Usa 'capas lineales' en vez de parámetros explícitos
    # Los parámetros se incializan automáticamente y se agregan
    # a los parámetros de la red
    self.fc1 = Linear(d0,d1)#8 entradas y 12 neuronas en capa oculta
    self.fc2 = Linear(d1,d0) # 8 neuronas en capa oculta 
    self.fc3 = Linear(d0,1)# 1 neurona en la salida
    
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)  
    return x

# Compilar el modelo de red neuronal

Ahora que el modelo está definido, podemos compilarlo. La compilación del modelo utiliza las bibliotecas numéricas eficientes llamadas backend, como TensorFlow o Theano. El backend elige automáticamente la mejor forma de representar la red para entrenarse y hacer predicciones para ejecutar en su hardware. Al compilar, debemos especificar algunas propiedades adicionales requeridas al entrenar la red. Recuerde que entrenar una red significa encontrar el mejor conjunto de pesos y sesgos para hacer predicciones para este problema.

Debemos especificar la función de pérdida (loss) que se usará para evaluar un conjunto de entrenamiento; tambien se debe especificar el optimizador para buscar los pesos óptimos de la red; y cualquier métrica que nos permita evaluar el desempeño  durante el entrenamiento. En este caso utilizaremos la pérdida logarítmica (logarithmic loss), que para un problema de **clasificación binaria** se define en Keras como **crosentropia binaria (binary_crossentropy)**. También utilizaremos el algoritmo de **descenso de gradiente** eficiente llamado **adam**. Finalmente, debido a que es un problema de clasificación, la métrica que vamos a recopilar e informar es la **precisión (accuracy)**.

In [8]:
# Establecemos loss, optimizador y metrica de evaluación
model= FFNN()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters())
m = nn.Sigmoid()# para la ultima capa


# Ajustar el modelo de red neuronal (Fit)

Hemos definido nuestro modelo y lo hemos compilado listo para poder realizar un cálculo eficiente. Ahora es el momento de ejecutar el modelo en algunos datos. Podemos entrenar nuestro modelo con nuestros datos cargados llamando a la función de ajuste **fit ()** en el modelo.

El proceso de entrenamiento se ejecutará para un número fijo de iteraciones a través del conjunto de datos llamado **épocas (epochs)**, que debemos especificar usando el argumento nb_epoch. También podemos establecer el número de instancias que se evalúan antes de que se realice una actualización de peso en la red, denominada **tamaño de lote (batch_size)** que es establecido con el argumento de **batch_size**. Para este problema ejecutaremos un pequeño número de épocas (150) y usaremos un tamaño de lote relativamente pequeño de 10. Una vez más, estos pueden ser seleccionados experimentalmente por prueba y error.

In [9]:
epochs=100
for epoch in range(epochs):
  for i, batch in enumerate(data):
    # reshape input and push data to device
    X,Y = batch[:,0:8], batch[:,8:].reshape(-1,1)

    # forward
    outputs = model(X)
    loss = criterion(m(outputs), Y)
    
    #backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (i+1) % 10 == 0:
      print(f'epoch {epoch+1}/{epochs}, step {i+1}/{len(data)}, loss= {loss.item():.4f}')


epoch 1/100, step 10/77, loss= 1.4753
epoch 1/100, step 20/77, loss= 0.8127
epoch 1/100, step 30/77, loss= 0.9478
epoch 1/100, step 40/77, loss= 0.7601
epoch 1/100, step 50/77, loss= 0.7030
epoch 1/100, step 60/77, loss= 0.7382
epoch 1/100, step 70/77, loss= 0.6629
epoch 2/100, step 10/77, loss= 0.5911
epoch 2/100, step 20/77, loss= 0.8680
epoch 2/100, step 30/77, loss= 0.8286
epoch 2/100, step 40/77, loss= 0.7273
epoch 2/100, step 50/77, loss= 0.6699
epoch 2/100, step 60/77, loss= 0.7619
epoch 2/100, step 70/77, loss= 0.5574
epoch 3/100, step 10/77, loss= 0.6865
epoch 3/100, step 20/77, loss= 0.5536
epoch 3/100, step 30/77, loss= 0.6538
epoch 3/100, step 40/77, loss= 0.7893
epoch 3/100, step 50/77, loss= 0.6450
epoch 3/100, step 60/77, loss= 0.6772
epoch 3/100, step 70/77, loss= 0.6999
epoch 4/100, step 10/77, loss= 0.5880
epoch 4/100, step 20/77, loss= 0.5178
epoch 4/100, step 30/77, loss= 0.7011
epoch 4/100, step 40/77, loss= 0.6072
epoch 4/100, step 50/77, loss= 0.6126
epoch 4/100,

# Evaluar el modelo de red neuronal 

Hemos entrenado nuestra red neuronal en **todo** el conjunto de datos y podemos evaluar el rendimiento de la red con el mismo conjunto de datos. Esto no se suele hacer, sino que esto solo **nos dará una idea de qué tan bien hemos modelado el conjunto de datos (por ejemplo, la precisión del entrenamiento), pero no tenemos idea de qué tan bien podría funcionar el algoritmo con los nuevos datos**. Lo hemos hecho para simplificar, pero lo ideal es que pueda separar sus datos en conjuntos de datos de entrenamiento y test, uno para el entrenamiento y otro para la evaluación de su modelo.

Puede evaluar su modelo en sus datos de entrenamiento utilizando la función evaluate() de su modelo y pasarle la misma entrada y salida utilizada para entrenar el modelo. Esto generará una predicción para cada par de entrada y salida, y recopilará el rendimiento de la red, incluida la pérdida promedio y cualquier métrica que haya configurado, como la precisión.

In [10]:
# evaluar el modelo
with torch.no_grad():
  n_correct = 0
  n_samples = 0
  for i, batch in enumerate(data):
    X,Y = batch[:,0:8], batch[:,8:].reshape(-1,1)
    outputs = model(X)
    # return the value and index (index = predictions)
    predicted = (m(outputs) > 0.5).float()
    n_correct += (predicted == Y).sum().item()
    n_samples += len(Y)

  accuracy = 100.0 * n_correct / n_samples
  print(f'accuracy = {accuracy}')


accuracy = 77.08333333333333


# Unir todo el proceso

Acaba de ver cómo puede crear fácilmente su primer modelo de red neuronal en Keras. Vamos a unirlo todo en un ejemplo de código completo.

In [11]:
from torch.nn import Sequential
from torch.nn import Linear
import numpy

seed = 7
numpy.random.seed(seed)

# datos
dataset_numpy = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
data =  DataLoader(torch.from_numpy(dataset_numpy).float(), batch_size=10, shuffle=True)

# modelo
model=Sequential(OrderedDict([
    ('fc1', nn.Linear(8,12)),
    ('relu', nn.ReLU()),
    ('fc2', nn.Linear(12,8)),
    ('relu', nn.ReLU()),
    ('fc3', nn.Linear(8,1))
    ]))
# compilar
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters())
m = nn.Sigmoid()
# ajustar
epochs=100
for epoch in range(epochs):
  for i, batch in enumerate(data):
    # reshape input and push data to device
    X,Y = batch[:,0:8], batch[:,8:].reshape(-1,1)

    # forward
    outputs = model(X)
    loss = criterion(m(outputs), Y)
    
    #backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (i+1) % 10 == 0:
      print(f'epoch {epoch+1}/{epochs}, step {i+1}/{len(data)}, loss= {loss.item():.4f}')

# evaluar el modelo
with torch.no_grad():
  n_correct = 0
  n_samples = 0
  for i, batch in enumerate(data):
    X,Y = batch[:,0:8], batch[:,8:].reshape(-1,1)
    outputs = model(X)
    # return the value and index (index = predictions)
    predicted = (m(outputs) > 0.5).float()
    n_correct += (predicted == Y).sum().item()
    n_samples += len(Y)

  accuracy = 100.0 * n_correct / n_samples
  print(f'accuracy = {accuracy}')

epoch 1/100, step 10/77, loss= 0.4809
epoch 1/100, step 20/77, loss= 0.9549
epoch 1/100, step 30/77, loss= 0.7022
epoch 1/100, step 40/77, loss= 0.6068
epoch 1/100, step 50/77, loss= 0.6172
epoch 1/100, step 60/77, loss= 1.2309
epoch 1/100, step 70/77, loss= 0.7388
epoch 2/100, step 10/77, loss= 0.8011
epoch 2/100, step 20/77, loss= 0.6338
epoch 2/100, step 30/77, loss= 0.7759
epoch 2/100, step 40/77, loss= 0.7431
epoch 2/100, step 50/77, loss= 0.8546
epoch 2/100, step 60/77, loss= 0.6923
epoch 2/100, step 70/77, loss= 0.8109
epoch 3/100, step 10/77, loss= 0.6221
epoch 3/100, step 20/77, loss= 0.6977
epoch 3/100, step 30/77, loss= 0.7975
epoch 3/100, step 40/77, loss= 0.6923
epoch 3/100, step 50/77, loss= 0.6310
epoch 3/100, step 60/77, loss= 0.5360
epoch 3/100, step 70/77, loss= 0.6239
epoch 4/100, step 10/77, loss= 0.8303
epoch 4/100, step 20/77, loss= 0.6405
epoch 4/100, step 30/77, loss= 0.8558
epoch 4/100, step 40/77, loss= 0.3234
epoch 4/100, step 50/77, loss= 0.5455
epoch 4/100,